# Getting Started

Agents use an LLM to determine which actions to take and in what order.
An action can either be using a tool and observing its output, or returning to the user.

When used correctly agents can be extremely powerful. The purpose of this notebook is to show you how to easily use agents through the simplest, highest level API.

In order to load agents, you should understand the following concepts:

- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. The interface for a tool is currently a function that is expected to have a string as an input, with a string as an output.
- LLM: The language model powering the agent.
- Agent: The agent to use. This should be a string that references a support agent class. Because this notebook focuses on the simplest, highest level API, this only covers using the standard supported agents.

**Agents**: For a list of supported agents and their specifications, see [here](https://python.langchain.com/docs/modules/agents/agent_types/).

**Tools**: For a list of predefined tools and their specifications, see [here](https://python.langchain.com/docs/modules/agents/tools/).

In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI
from langchain.tools import Tool

First, let's load the language model we're going to use to control the agent.

In [2]:
llm = OpenAI(temperature=0)

Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.

In [3]:
tools = load_tools(
    ["serpapi"], # tool for web search
    llm=llm
)

We will add a tool that the agent can use to add two numbers

In [4]:
def add_numbers(str_numbers):
    num1, num2 = str_numbers.split(',')[:2]
    return float(num1) + float(num2)

In [5]:
tools.append(
    Tool.from_function(
        func=add_numbers,
        name = "addTool",
        description="Used to add two numbers. Takes two numbers seperated by a comma as input."
    )
)

In [8]:
agent = initialize_agent(
    tools, 
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=3
)

Now let's test it out!

In [9]:
agent.run("Determine what the current temperature in Oxon Hill Maryland is plus 10 degrees.")



> Entering new AgentExecutor chain...
 I need to find the current temperature and then add 10 degrees.
Action: Search
Action Input: Current temperature in Oxon Hill Maryland
Observation: Partly cloudy in the morning followed by scattered thunderstorms in the afternoon. High 87F. Winds WSW at 5 to 10 mph. Chance of rain 40%.
Thought: I need to add 10 degrees to the current temperature.
Action: addTool
Action Input: 87, 10
Observation: 97.0
Thought: I now know the final answer.
Final Answer: The current temperature in Oxon Hill Maryland plus 10 degrees is 97.0 degrees.

> Finished chain.


'The current temperature in Oxon Hill Maryland plus 10 degrees is 97.0 degrees.'